In [1]:
import numpy as np
import pandas as pd

import nltk
import re
from nltk.tokenize import word_tokenize,sent_tokenize
#wt = word_tokenize()
#st = sent_tokenize()

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lt = WordNetLemmatizer()


from nltk.corpus import stopwords
s_words = stopwords.words('english')
import string
pun = list(string.punctuation)
s_words += pun


In [2]:
path = r'C:\Users\YATISH\Desktop\poems_classifier\data_3.csv'

data = pd.read_table(path, delimiter = ',')
df = data.copy()

In [3]:
poems = df.iloc[:,1]
cat_ = df.iloc[:,4]

In [4]:
from nltk.corpus import wordnet
def get_simple_pos (tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [5]:
def clean_text(words):
    output_words=[]
    for w in words:
        if w not in s_words:
            w = w.replace(r'\S+@\S+', ' ')  # remove email
            w = re.sub('[^a-z,A-Z]',' ',w)
            pos = pos_tag(w)
            clean_w = lt.lemmatize(w,pos =get_simple_pos(pos[0][1]))
            #clean_w = ps.stem(w)
            clean_w = clean_w.strip()  # remove trailing leading spaces
            clean_w = clean_w.replace(r'\s+', '') # remove whitespace
            clean_w = clean_w.lower()
            output_words.append(clean_w)
    return output_words

In [6]:
X=[]
Y=[]
for i in range(len(poems)):
    poems[i].replace("\n"," ").replace("\r"," ")
    words = word_tokenize(poems[i])
    X.append(clean_text(words))
    Y.append(cat_[i])

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.5)


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 50000,tokenizer=lambda doc: doc, lowercase=False)

In [17]:
X_train= cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [18]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [19]:
y_pred = clf.predict(X_test)
clf.score(X_test,Y_test)

0.5853658536585366

In [13]:
from sklearn.svm import SVC
clf2 = SVC()
clf2.fit(X_train,Y_train)

C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [14]:
y_pred = clf2.predict(X_test)
clf2.score(X_test,Y_test)

0.5347222222222222